In [ ]:
import sys as sys
import unyt
import h5py
import logging
import numpy as np
import pandas as pd
import time

logging.basicConfig(level = logging.INFO)
import readgadget
from spreadmetric import SpreadMetricCalculator
from simbaahf import SIMBASnapshotData
from holder import SimulationData
from transfer.data import ParticleData
import pickle

#my method for loading in the 1P sims with the new naming convention
list_of_strings = ["1P_1_n5", "1P_1_n4", "1P_1_n3", "1P_1_n2", "1P_1_n1", "1P_1_0", "1P_1_1", "1P_1_2", "1P_1_3",
                  "1P_1_4", "1P_1_5", "1P_2_n5", "1P_2_n4", "1P_2_n3", "1P_2_n2", "1P_2_n1", "1P_2_0", "1P_2_1", 
                   "1P_2_2", "1P_2_3", "1P_2_4", "1P_2_5", "1P_3_n5", "1P_3_n4", "1P_3_n3", "1P_3_n2", "1P_3_n1", 
                   "1P_3_0", "1P_3_1", "1P_3_2", "1P_3_3", "1P_3_4", "1P_3_5", "1P_4_n5", "1P_4_n4", "1P_4_n3", 
                   "1P_4_n2", "1P_4_n1", "1P_4_0", "1P_4_1", "1P_4_2", "1P_4_3", "1P_4_4", "1P_4_5", "1P_5_n5", 
                   "1P_5_n4", "1P_5_n3", "1P_5_n2", "1P_5_n1", "1P_5_0", "1P_5_1", "1P_5_2", "1P_5_3","1P_5_4", 
                   "1P_5_5", "1P_6_n5", "1P_6_n4", "1P_6_n3", "1P_6_n2", "1P_6_n1", "1P_6_0", "1P_6_1", "1P_6_2", 
                   "1P_6_3", "1P_6_4", "1P_6_5",]
string_array = []
for i in range(7,29):
    strings = [f"1P_{i}_n2", f"1P_{i}_n1",  f"1P_{i}_1", f"1P_{i}_2"]
    string_array = np.append(string_array, strings)

def simba_spread(suite, sim):
    start = time.time()

    simdir = f'/home/jovyan/PUBLIC_RELEASE/Sims/{suite}/{sim[:2]}/{sim}/'    #exact directory depends on which binder you're in

    initial_filename = simdir + 'ICs/ics' 
    final_filename = simdir + f'snap_033.hdf5'

    initial_halo_filename = None
    final_halo_filename = f'/home/jovyan/PUBLIC_RELEASE/FOF_Subfind/{suite}/{sim[:2]}/{sim}/'+ f'fof_subhalo_tab_033.hdf5'

    initial = SIMBASnapshotData(initial_filename, initial_halo_filename)

    truncate_ids = {
        0: initial.gas.particle_ids.max() + 1,
        1: None,
        4: initial.gas.particle_ids.max() + 1,
    }


    final = SIMBASnapshotData(
        final_filename, final_halo_filename, truncate_ids=truncate_ids
    )

    cross = SimulationData(initial_snapshot=initial, final_snapshot=final)

    spread_metric = SpreadMetricCalculator(simulation=cross)


    spread_metric.find_neighbours()
    spread_metric.find_neighbour_distances()

    with h5py.File(final_filename, "r") as handle:
        dark_matter_final_ids = unyt.unyt_array(handle["PartType1/ParticleIDs"][:], units=None)
        gas_final_ids = unyt.unyt_array(handle["PartType0/ParticleIDs"][:], units=None)
        gas_final_coordinates = unyt.unyt_array(handle["PartType0/Coordinates"][:], units=None)
        gas_final_masses = unyt.unyt_array(handle["PartType0/Masses"][:], units=None)
        
    print("Creating output directories")
    
    neighbors_file = f'/home/jovyan/home/spread_metric/{sim}/neighbors.hdf5'
    neighbors = h5py.File(neighbors_file, 'r')
    
    spread_metric_dark_matter_initial_neighbors = neighbors['dark_matter/initial_neighbors'][:]
    spread_metric_dark_matter_final_neighbors = neighbors['dark_matter/final_neighbors'][:]
    spread_metric_gas_initial_neighbors = neighbors['gas/initial_neighbors'][:]
    spread_metric_gas_final_neighbors = neighbors['gas/final_neighbors'][:]

    dmi_IDs = pd.DataFrame(dark_matter_final_ids, columns = ["IDs"])
    dmf_IDs = pd.DataFrame(spread_metric.dark_matter_final_ids, columns = ["IDs"])

    dmi_IDs['index'] =dmi_IDs.index 
    dmi_IDs = dmi_IDs.set_index('IDs')
    dmi_IDs = dmi_IDs.reindex(index=dmf_IDs['IDs'])
    dmi_IDs = dmi_IDs.reset_index()

    dm_indices = dmi_IDs['index'].tolist()

    gas_final_ids, indicies = np.unique(gas_final_ids, return_index=True)
    gas_final_coordinates = gas_final_coordinates[indicies]
    gas_final_masses = gas_final_masses[indicies]

    gasi_IDs = pd.DataFrame(gas_final_ids, columns = ["IDs"])
    gasf_IDs = pd.DataFrame(spread_metric.gas_final_ids, columns = ["IDs"])

    gasi_IDs['index'] =gasi_IDs.index 
    gasi_IDs = gasi_IDs.set_index('IDs')
    gasi_IDs = gasi_IDs.reindex(index=gasf_IDs['IDs'])
    gasi_IDs = gasi_IDs.reset_index()

    gas_indices = gasi_IDs['index'].tolist()
    
    ### Matching the other way ###
    
    dmi_IDs = pd.DataFrame(spread_metric.dark_matter_final_ids, columns = ["IDs"])
    dmf_IDs = pd.DataFrame(dark_matter_final_ids, columns = ["IDs"])

    dmi_IDs['index'] =dmi_IDs.index 
    dmi_IDs = dmi_IDs.set_index('IDs')
    dmi_IDs = dmi_IDs.reindex(index=dmf_IDs['IDs'])
    dmi_IDs = dmi_IDs.reset_index()

    dm_indices2 = dmi_IDs['index'].tolist()

    print("Matching gas arrays")

    gas_final_ids, indicies = np.unique(gas_final_ids, return_index=True)

    gasi_IDs = pd.DataFrame(spread_metric.gas_final_ids, columns = ["IDs"])
    gasf_IDs = pd.DataFrame(gas_final_ids, columns = ["IDs"])

    gasi_IDs['index'] = gasi_IDs.index 
    gasi_IDs = gasi_IDs.set_index('IDs')
    gasi_IDs = gasi_IDs.reindex(index=gasf_IDs['IDs'])
    gasi_IDs = gasi_IDs.reset_index()

    gas_indices2 = gasi_IDs['index'].tolist()


    hf = h5py.File(f'/home/jovyan/home/spread_metric/{suite}/{sim}/Spread_Output_{sim}_snap033.hdf5', 'w')

    dm = hf.create_group('dark_matter')
    gas = hf.create_group('gas')
    #star = hf.create_group('stars')

    dm.create_dataset('spread', data = spread_metric.dark_matter_spread)
    dm.create_dataset('halo_IDs', data = cross.dark_matter_haloes)
    dm.create_dataset('lagrangian_IDs', data = cross.dark_matter_lagrangian_regions)
    dm.create_dataset('snapshot_to_output_index', data = dm_indices)
    dm.create_dataset('output_to_snapshot_index', data = dm_indices2)
    dm.create_dataset('initial_coordinates', data = spread_metric.dark_matter_initial_coordinates)
    dm.create_dataset('final_coordinates', data = spread_metric.dark_matter_final_coordinates)
    dm.create_dataset('initial_IDs', data = spread_metric.dark_matter_initial_ids)
    dm.create_dataset('final_IDs', data = spread_metric.dark_matter_final_ids)
    dm.create_dataset('initial_neighbors', data = spread_metric_dark_matter_initial_neighbors)
    dm.create_dataset('final_neighbors', data = spread_metric_dark_matter_final_neighbors)

    gas.create_dataset('spread', data = spread_metric.gas_spread)
    gas.create_dataset('halo_IDs', data = cross.gas_haloes)
    gas.create_dataset('lagrangian_IDs', data = cross.gas_lagrangian_regions)
    gas.create_dataset('snapshot_to_output_index', data = gas_indices)
    gas.create_dataset('output_to_snapshot_index', data = gas_indices2)
    gas.create_dataset('initial_coordinates', data = spread_metric.gas_initial_coordinates)
    gas.create_dataset('final_coordinates', data = spread_metric.gas_final_coordinates)
    gas.create_dataset('initial_IDs', data = spread_metric.gas_initial_ids)
    gas.create_dataset('final_IDs', data = spread_metric.gas_final_ids)
    gas.create_dataset('initial_neighbors', data = spread_metric_gas_initial_neighbors)
    gas.create_dataset('final_neighbors', data = spread_metric_gas_final_neighbors)

    #star.create_dataset('Spread', data = spread_metric.star_spread)
    #star.create_dataset('HaloIDs', data = cross.star_haloes)
    #star.create_dataset('LagrangianIDs', data = cross.star_lagrangian_regions)
    #star.create_dataset('Snapshot2OutputIndex', data = star_indices)
    #star.create_dataset('final_coordinates', data = spread_metric.star_final_coordinates)
    #star.create_dataset('final_IDs', data = spread_metric.star_final_ids)

    hf.close()

    end = time.time()

    elapsed = end-start

    print(f'it took {elapsed} seconds to run')


In [ ]:
#need directories for the files to be saved in
#here's an example for the 1P set

suite = "SIMBA"
import os
string_array = []
for i in range(7,29):
    strings = [f"1P_{i}_0",  f"1P_{i}_1"]
    string_array = np.append(string_array, strings)

for sim in string_array:
    path = f'/home/jovyan/home/spread_metric/{suite}/{sim}'
    os.mkdir(path)

In [ ]:
#Loops over simulations and runs the spread driver for each

for i in range(len(string_array)):
    simba_spread("SIMBA", string_array[i])